<a href="https://colab.research.google.com/github/BenWilop/WSG_games/blob/main/playground_WSG_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import dotenv
import wandb
import os

dotenv.load_dotenv(os.path.join("/homes/55/bwilop/wsg/private/", "vscode-ssh.env"))
api_key = os.getenv("WANDB_API_KEY")
wandb.login(key=api_key)
WANDB_ENTITIY = "benwilop-rwth-aachen-university"

data_folder = "/homes/55/bwilop/wsg/data/"
experiment_folder = "/homes/55/bwilop/wsg/experiments/"
crosscoder_folder = experiment_folder + "tictactoe/crosscoder/"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /homes/55/bwilop/.netrc
wandb: Currently logged in as: benwilop (benwilop-rwth-aachen-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
%load_ext autoreload
%autoreload 2
import json
import torch as t

from jaxtyping import Float
import matplotlib.pyplot as plt

from wsg_games.meta import *

from wsg_games.tictactoe.evals import *
from wsg_games.tictactoe.data import *
from wsg_games.tictactoe.game import *

from wsg_games.tictactoe.analysis.analyse_data import *
from wsg_games.tictactoe.analysis.visualize_game import *

from wsg_games.tictactoe.train.create_models import *
from wsg_games.tictactoe.train.save_load_models import *
from wsg_games.tictactoe.train.train import *
from wsg_games.tictactoe.train.finetune import *
from wsg_games.tictactoe.train.pretrain import *

DEVICE = t.device("cuda" if t.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# Load Data & Models

In [3]:
# project_name_pretrain = "tictactoe/tictactoe_pretraining5"
# project_name_finetune = "tictactoe/tictactoe_finetuning5"
project_name_pretrain = "tictactoe/tictactoe_pretraining6"
project_name_finetune = "tictactoe/tictactoe_finetuning_use_best_val_step6_lre5"
weak_model_size = "medium"
strong_model_size = "huge"
index = 2

# Load data
(
    tictactoe_train_data,
    tictactoe_weak_finetune_data,
    tictactoe_val_data,
    tictactoe_test_data,
) = load_split_data(data_folder + "tictactoe/", device=DEVICE, index=index)

# Load models
weak_model = load_model(
    project_name_pretrain,
    weak_model_size,
    Goal.WEAK_GOAL,
    experiment_folder,
    device=DEVICE,
    index=index,
)
strong_baseline_model = load_model(
    project_name_pretrain,
    strong_model_size,
    Goal.WEAK_GOAL,
    experiment_folder,
    device=DEVICE,
    index=index,
)
strong_model = load_model(
    project_name_pretrain,
    strong_model_size,
    Goal.STRONG_GOAL,
    experiment_folder,
    device=DEVICE,
    index=index,
)
finetuned_model = load_finetuned_model(
    project_name_finetune,
    weak_model_size,
    strong_model_size,
    experiment_folder,
    DEVICE,
    index,
)

# Print evaluations
(
    weak_loss,
    _,
) = quick_evaluation("### weak_model ### ", weak_model, tictactoe_test_data)
strong_baseline_loss, _ = quick_evaluation(
    "###  strong_baseline_model ### ", strong_baseline_model, tictactoe_test_data
)
quick_evaluation("### strong_model ### ", strong_model, tictactoe_test_data)
weak_finetuned_loss, _ = quick_evaluation(
    "###  finetuned_model ### ", finetuned_model, tictactoe_test_data
)
print(
    "Performance Gap Recovered (PGR): ",
    (weak_loss - weak_finetuned_loss) / (weak_loss - strong_baseline_loss),
)

experiment_folder:  /homes/55/bwilop/wsg/experiments/
project_name:  tictactoe/tictactoe_pretraining6
/homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6
Loading model from /homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6/experiment_2_medium_weak_2025-05-26-03-18_swx5asc6.pkl
Moving model to device:  cuda
experiment_folder:  /homes/55/bwilop/wsg/experiments/
project_name:  tictactoe/tictactoe_pretraining6
/homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6
Loading model from /homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6/experiment_2_huge_weak_2025-05-26-03-24_akjxqang.pkl
Moving model to device:  cuda
experiment_folder:  /homes/55/bwilop/wsg/experiments/
project_name:  tictactoe/tictactoe_pretraining6
/homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6
Loading model from /homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6/experiment_2_huge_strong_2025-05-26-04-02_twn0w0nj.pkl
Moving model to de

In [4]:
from dictionary_learning.dictionary_learning.cache import *
from wsg_games.tictactoe.crosscoder.collect_activations import *
from wsg_games.tictactoe.crosscoder.crosscoder_metrics import *
from wsg_games.tictactoe.crosscoder.train_crosscoder import *
from wsg_games.tictactoe.crosscoder.crosscoder_feature_explorer import *

In [5]:
# Strong
strong_train_activations_path, strong_val_activations_path = compute_activations(
    Game.TICTACTOE,
    Goal.STRONG_GOAL,
    project_name_pretrain,
    None,
    None,
    strong_model_size,
    index,
    crosscoder_folder,
    tictactoe_test_data.games_data,
    tictactoe_val_data.games_data,
    experiment_folder,
    device=DEVICE,
)

# Finetuned
finetuned_train_activations_path, finetuned_val_activations_path = compute_activations(
    Game.TICTACTOE,
    None,
    None,
    weak_model_size,
    project_name_finetune,
    strong_model_size,
    index,
    crosscoder_folder,
    tictactoe_test_data.games_data,
    tictactoe_val_data.games_data,
    experiment_folder,
    device=DEVICE,
)

print("strong_train_activations_path", strong_train_activations_path)
print("strong_val_activations_path:", strong_val_activations_path)
print("finetuned_train_activations_path:", finetuned_train_activations_path)
print("finetuned_val_activations_path:", finetuned_val_activations_path)

# Verify

for layer in range(4):
    validate_activations(
        [
            strong_train_activations_path + f"/layer_{layer}_out",
            finetuned_train_activations_path + f"/layer_{layer}_out",
        ]
    )
    validate_activations(
        [
            strong_val_activations_path + f"/layer_{layer}_out",
            finetuned_val_activations_path + f"/layer_{layer}_out",
        ]
    )

experiment_folder:  /homes/55/bwilop/wsg/experiments/
project_name:  tictactoe/tictactoe_pretraining6
/homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6
Loading model from /homes/55/bwilop/wsg/experiments/tictactoe/tictactoe_pretraining6/experiment_2_huge_strong_2025-05-26-04-02_twn0w0nj.pkl
Moving model to device:  cuda
Shards already exist in /homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/activations/2_huge_strong_train
Instead, you can set overwrite=True to overwrite existing shards.
Shards already exist in /homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/activations/2_huge_strong_val
Instead, you can set overwrite=True to overwrite existing shards.
Moving model to device:  cuda
Shards already exist in /homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/activations/2_huge_finetuned_through_medium_train
Instead, you can set overwrite=True to overwrite existing shards.
Shards already exist in /homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/activation

In [6]:
layer = 3  # 0 indexed

model_0_name = "strong_model"
model_1_name = "finetuned_model"

train_activations_stor_dir_model_0 = get_activations_path(
    Goal.STRONG_GOAL, None, strong_model_size, index, crosscoder_folder, "train"
)
val_activations_stor_dir_model_0 = get_activations_path(
    Goal.STRONG_GOAL, None, strong_model_size, index, crosscoder_folder, "val"
)
train_activations_stor_dir_model_1 = get_activations_path(
    None, weak_model_size, strong_model_size, index, crosscoder_folder, "train"
)
val_activations_stor_dir_model_1 = get_activations_path(
    None, weak_model_size, strong_model_size, index, crosscoder_folder, "val"
)
training_cfg_cross_coder = get_training_cfg_cross_coder()
train_crosscoder(
    model_0_name,
    model_1_name,
    index,
    train_activations_stor_dir_model_0 + f"/layer_{layer}_out",
    val_activations_stor_dir_model_0 + f"/layer_{layer}_out",
    train_activations_stor_dir_model_1 + f"/layer_{layer}_out",
    val_activations_stor_dir_model_1 + f"/layer_{layer}_out",
    layer,
    training_cfg_cross_coder,
    WANDB_ENTITIY,
    DEVICE,
    crosscoder_folder,
)

Training on 1107968 token activations.
Validating on 553536 token activations.
Activation dim: 128
Dictionary size: 512


/homes/55/bwilop/wsg/WSG_games/dictionary_learning/dictionary_learning/training.py:239: UserWarning: Error saving config: 'generator' object has no attribute 'config'
  warn(f"Error saving config: {e}")
  5%|▍         | 985/20000 [00:06<01:51, 171.03it/s]

Validating at step 1000


 10%|▉         | 1990/20000 [00:53<01:50, 162.36it/s] 

Validating at step 2000


...

Validating at step 18000


 95%|█████████▍| 18992/20000 [14:13<00:05, 179.31it/s]

Validating at step 19000


100%|██████████| 554/554 [00:50<00:00, 11.07it/s]


step,▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇████
train/auxk_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███
train/cl0_frac_variance_explained,▁▃▃▇▇▇▇▇▇██████▇████████████████████████
train/cl1_frac_variance_explained,▁▂▁▄▆▅▃█▅▄▆▇▆▆▇▇▇▇▆█▇█▇▇▇█▇▆██▇▇▇█▇█▇███
train/effective_l0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/frac_deads,▅▄▆█▄▅▄▅▄▃▃▃▆█▃▂▃▇▃▅▁▂▄▄▂▂▃▂▄▂▄▄▅▅▄▄▅▃▁▄
train/frac_variance_explained,▁▇▇█████████████████████████████████████
train/l0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/l2_loss,█▄▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▂▂▂▂▁▂▂▂▁▂▂▁▂▂▁
train/loss,█▅▄▂▃▃▂▂▃▂▃▂▃▂▂▂▃▂▁▂▂▁▂▁▁▂▃▂▁▁▂▁▃▁▁▂▂▂▁▂
train/mse_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [7]:
raise Exception()

Exception: 

In [ ]:
# save_dir = "/homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/checkpoints/experiment_2_strong_model_finetuned_model_2025-05-31-10-35"
# save_dir = "/homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/checkpoints/experiment_2_strong_model_finetuned_model_2025-06-03-11-28"
# save_dir = "/homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/checkpoints/experiment_2_strong_model_finetuned_model_2025-06-03-17-28"

# Layer 0
# save_dir = "/homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/checkpoints/experiment_2_strong_model_finetuned_model_2025-06-03-19-56"
# Layer 3
save_dir = "/homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/checkpoints/experiment_2_strong_model_finetuned_model_2025-06-03-21-05"
# Layer 5
# save_dir = "/homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/checkpoints/experiment_2_strong_model_finetuned_model_2025-06-03-19-11"

crosscoder_metrics = CrosscoderMetrics.cache(save_dir, DEVICE, overwrite=True)
# crosscoder_metrics = CrosscoderMetrics(save_dir, DEVICE)

File not found. Creating new CrosscoderMetrics ...


In [ ]:
predictions = crosscoder_metrics._get_or_compute_model_predictions(
    weak_model, strong_model, finetuned_model
)

Pre-computing model predictions for all features. This may take a while...


Pre-computing predictions:   0%|          | 0/512 [00:00<?, ?it/s]

Saving computed predictions to /homes/55/bwilop/wsg/experiments/tictactoe/crosscoder/checkpoints/experiment_2_strong_model_finetuned_model_2025-06-03-19-56/crosscoder_metrics_model_predictions.pkl...


In [ ]:
explorer = CrosscoderFeatureExplorer(crosscoder_metrics)
explorer.display()
# 24 -> complete X
# 15, 21, 25, 37, 38  -> complete X or block O
# 27, 30, 34 -> complete O or block X

# 35